# AWS Step Functions Data Science SDK를 사용하여 엔드 투 엔드 Amazon Personalize 모델 배포 프로세스를 운영합니다.

1. [소개](#Introduction)
2. [설정](#Setup)
3. [작업-상태](#Task-States)
4. [대기-상태](#Wait-States)
5. [선택-상태](#Choice-States)
6. [워크플로](#Workflow)
7. [생성-추천](#Generate-Recommendations)



## 소개

이 노트북에서는 AWS Step Functions Data Science SDK를 사용하여 Amazon Personalize 워크플로를 만들고 관리하는 방법을 설명합니다. 이 Step Functions SDK는 데이터 사이언티스트가 AWS Step Functions를 사용하여 기계 학습 워크플로를 손쉽게 만들고 실행할 수 있는 오픈 소스 라이브러리입니다. Step Functions SDK에 대한 자세한 내용은 다음을 참조하세요.
* [AWS Step Functions](https://aws.amazon.com/step-functions/)
* [AWS Step Functions 개발자 가이드](https://docs.aws.amazon.com/step-functions/latest/dg/welcome.html)
* [AWS Step Functions Data Science SDK](https://aws-step-functions-data-science-sdk.readthedocs.io)

이 노트북에서는 이 SDK를 사용하여 Personalize 리소스를 생성하는 단계를 만들고, 리소스를 하나로 연결하여 워크플로를 만들고, AWS Step Functions에서 워크플로를 실행합니다.

Amazon Personalize에 대한 자세한 내용은 다음을 참조하세요.

* [Amazon Personalize](https://aws.amazon.com/personalize/)


## 설정

### SDK에서 필요한 모듈 가져오기

In [ ]:
#import sys
#!{sys.executable} -m pip install --upgrade stepfunctions

In [ ]:
import boto3
import json
import numpy as np
import pandas as pd
import time

personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')


import stepfunctions
import logging

from stepfunctions.steps import *
from stepfunctions.workflow import Workflow

stepfunctions.set_stream_logger(level=logging.INFO)

workflow_execution_role = "<Workflow exection role name>" # paste the StepFunctionsWorkflowExecutionRole ARN from above

### S3 위치 및 파일 이름 설정

In [ ]:
bucket = "<Bucket Name>"       # replace with the name of your S3 bucket
filename = "<File Name>"  # replace with a name that you want to save the dataset under

### IAM 역할 설정

#### Step Functions를 위한 실행 역할 생성

Step Functions에서 워크플로를 생성하고 실행할 수 있는 실행 역할이 필요합니다.

1. [IAM 콘솔](https://console.aws.amazon.com/iam/)로 이동합니다.
2. **역할(Roles)**을 선택한 후 **역할 생성(Create role)**을 선택합니다.
3. **이 역할을 사용할 서비스 선택(Choose the service that will use this role)**에서 **Step Functions**를 선택합니다.
4. **역할 이름(Role name)**을 입력할 수 있을 때까지 **다음(Next)**을 선택합니다.
5. `StepFunctionsWorkflowExecutionRole`과 같은 이름을 입력한 후 **역할 생성(Create role)**을 선택합니다.


생성한 역할에 정책을 연결합니다. 다음 단계에서는 Step Functions에 대한 모든 권한을 제공하는 정책을 연결합니다. 단, 필요한 리소스에 대한 액세스 권한만 제공하는 것이 좋습니다.

1. **권한(Permissions)** 탭에서 **인라인 정책 추가(Add inline policy)**를 클릭합니다.
2. **JSON** 탭에 다음을 입력합니다.

```json
{
    "Version": "2012-10-17",
    "Statement": [
    
        {
            "Effect": "Allow",
            "Action": [
                "personalize:*"
            ],
            "Resource": "*"
        },   

        {
            "Effect": "Allow",
            "Action": [
                "lambda:InvokeFunction"
            ],
            "Resource": "*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "iam:PassRole"
            ],
            "Resource": "*",
        },
        {
            "Effect": "Allow",
            "Action": [
                "events:PutTargets",
                "events:PutRule",
                "events:DescribeRule"
            ],
            "Resource": "*"
        }
    ]
}
```

3. **정책 검토(Review policy)**를 선택하고 정책에 `StepFunctionsWorkflowExecutionPolicy`와 같은 이름을 지정합니다.
4. **정책 생성(Create policy)**을 선택합니다. 역할의 세부 정보 페이지로 리디렉션됩니다.
5. **요약(Summary)**의 상단에서 **역할 ARN(Role ARN)**을 복사합니다.



In [ ]:
lambda_state_role = LambdaStep(
    state_id="create bucket and role",
    parameters={  
        "FunctionName": "stepfunction_create_personalize_role", #replace with the name of the function you created
        "Payload": {  
           "bucket": bucket
        }
    },
    result_path='$'
 
)

lambda_state_role.add_retry(Retry(
    error_equals=["States.TaskFailed"],
    interval_seconds=5,
    max_attempts=1,
    backoff_rate=4.0
))

lambda_state_role.add_catch(Catch(
    error_equals=["States.TaskFailed"],
    next_step=Fail("CreateRoleTaskFailed")
))

#### S3 버킷에 정책 연결

In [ ]:
s3 = boto3.client("s3")

policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket),
                "arn:aws:s3:::{}/*".format(bucket)
                
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket, Policy=json.dumps(policy))

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role


#### Personalize 역할 생성


In [ ]:
iam = boto3.client("iam")

role_name = "<Role Name>" # Create a personalize role


assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)



policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]


## 데이터-준비

### 훈련 데이터 다운로드, 준비 및 업로드

In [ ]:
!pwd

In [ ]:
!wget -N http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -o ml-100k.zip
data = pd.read_csv('./ml-100k/u.data', sep='\t', names=['USER_ID', 'ITEM_ID', 'RATING', 'TIMESTAMP'])
pd.set_option('display.max_rows', 5)
data



In [ ]:
data = data[data['RATING'] > 2]                # keep only movies rated 2 and above
data2 = data[['USER_ID', 'ITEM_ID', 'TIMESTAMP']] 
data2.to_csv(filename, index=False)

boto3.Session().resource('s3').Bucket(bucket).Object(filename).upload_file(filename)

## 작업-상태

### Lambda 작업 상태

Step Functions에서 `Task` 상태는 워크플로에 의해 수행되는 단일 작업 단위를 나타냅니다. 작업은 Lambda 함수를 호출하고 다른 AWS 서비스를 오케스트레이션할 수 있습니다. AWS Step Functions 개발자 가이드에서 [AWS 서비스 통합](https://docs.aws.amazon.com/step-functions/latest/dg/concepts-service-integrations.html)을 참조하세요.

다음은 `lambda_state`라는 [LambdaStep](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/compute.html#stepfunctions.steps.compute.LambdaStep)을 생성한 다음 Lambda 함수가 실패할 경우에 적용할 재시도 옵션을 구성합니다.

#### Lambda 함수 생성

이 워크플로의 Lambda 작업 상태는 스키마, 데이터 세트 그룹, 데이터 세트, 솔루션, 솔루션 버전 등의 Personalize 리소스를 반환하는 Lambda 함수**(Python 3.x)**를 사용합니다. [Lambda 콘솔](https://console.aws.amazon.com/lambda/)에서 다음 함수를 생성합니다.

1. stepfunction-create-schema
2. stepfunctioncreatedatagroup
3. stepfunctioncreatedataset
4. stepfunction-createdatasetimportjob
5. stepfunction_select-recipe_create-solution
6. stepfunction_create_solution_version
7. stepfunction_getsolution_metric_create_campaign

./Lambda/ 폴더에서 해당 Lambda 함수 코드를 복사해 붙여 넣습니다.


#### 스키마 생성

In [ ]:
lambda_state_schema = LambdaStep(
    state_id="create schema",
    parameters={  
        "FunctionName": "stepfunction-create-schema", #replace with the name of the function you created
        "Payload": {  
           "input": "personalize-stepfunction-schema263"
        }
    },
    result_path='$'    
)

lambda_state_schema.add_retry(Retry(
    error_equals=["States.TaskFailed"],
    interval_seconds=5,
    max_attempts=1,
    backoff_rate=4.0
))

lambda_state_schema.add_catch(Catch(
    error_equals=["States.TaskFailed"],
    next_step=Fail("CreateSchemaTaskFailed")
))

#### 데이터 세트 그룹 생성

In [ ]:
lambda_state_datasetgroup = LambdaStep(
    state_id="create dataset Group",
    parameters={  
        "FunctionName": "stepfunctioncreatedatagroup", #replace with the name of the function you created
        "Payload": {  
           "input": "personalize-stepfunction-dataset-group", 
           "schemaArn.$": '$.Payload.schemaArn'
        }
    },

    result_path='$'
)



lambda_state_datasetgroup.add_retry(Retry(
    error_equals=["States.TaskFailed"],
    interval_seconds=5,
    max_attempts=1,
    backoff_rate=4.0
))


lambda_state_datasetgroup.add_catch(Catch(
    error_equals=["States.TaskFailed"],
    next_step=Fail("CreateDataSetGroupTaskFailed")
))

#### 데이터 세트 생성

In [ ]:
lambda_state_createdataset = LambdaStep(
    state_id="create dataset",
    parameters={  
        "FunctionName": "stepfunctioncreatedataset", #replace with the name of the function you created
#        "Payload": {  
#           "schemaArn.$": '$.Payload.schemaArn',
#           "datasetGroupArn.$": '$.Payload.datasetGroupArn',
            
            
#        }
        
        "Payload": {  
           "schemaArn.$": '$.schemaArn',
           "datasetGroupArn.$": '$.datasetGroupArn',        
        } 
        
        
    },
    result_path = '$'
)

lambda_state_createdataset.add_retry(Retry(
    error_equals=["States.TaskFailed"],
    interval_seconds=5,
    max_attempts=1,
    backoff_rate=4.0
))

lambda_state_createdataset.add_catch(Catch(
    error_equals=["States.TaskFailed"],
    next_step=Fail("CreateDataSetTaskFailed")
))

#### 데이터 세트 가져오기 작업 생성

In [ ]:
lambda_state_datasetimportjob = LambdaStep(
    state_id="create dataset import job",
    parameters={  
        "FunctionName": "stepfunction-createdatasetimportjob", #replace with the name of the function you created
        "Payload": {  
           "datasetimportjob": "stepfunction-createdatasetimportjob",
           "dataset_arn.$": '$.Payload.dataset_arn',
           "datasetGroupArn.$": '$.Payload.datasetGroupArn',
           "bucket_name": bucket,
           "file_name": filename,
           "role_arn": role_arn
            
        }
    },

    result_path = '$'
)

lambda_state_datasetimportjob.add_retry(Retry(
    error_equals=["States.TaskFailed"],
    interval_seconds=5,
    max_attempts=1,
    backoff_rate=4.0
))

lambda_state_datasetimportjob.add_catch(Catch(
    error_equals=["States.TaskFailed"],
    next_step=Fail("DatasetImportJobTaskFailed")
))

#### 레시피 및 솔루션 생성

In [ ]:
lambda_state_select_receipe_create_solution = LambdaStep(
    state_id="select receipe and create solution",
    parameters={  
        "FunctionName": "stepfunction_select-recipe_create-solution", #replace with the name of the function you created
        "Payload": {  
           #"dataset_group_arn.$": '$.Payload.datasetGroupArn' 
            "dataset_group_arn.$": '$.datasetGroupArn'
        }
    },
    result_path = '$'
)

lambda_state_select_receipe_create_solution.add_retry(Retry(
    error_equals=["States.TaskFailed"],
    interval_seconds=5,
    max_attempts=1,
    backoff_rate=4.0
))

lambda_state_select_receipe_create_solution.add_catch(Catch(
    error_equals=["States.TaskFailed"],
    next_step=Fail("DatasetReceiptCreateSolutionTaskFailed")
))

#### 솔루션 버전 생성

In [ ]:
lambda_create_solution_version = LambdaStep(
    state_id="create solution version",
    parameters={  
        "FunctionName": "stepfunction_create_solution_version", 
        "Payload": {  
           "solution_arn.$": '$.Payload.solution_arn'           
        }
    },
    result_path = '$'
)

lambda_create_solution_version.add_retry(Retry(
    error_equals=["States.TaskFailed"],
    interval_seconds=5,
    max_attempts=1,
    backoff_rate=4.0
))

lambda_create_solution_version.add_catch(Catch(
    error_equals=["States.TaskFailed"],
    next_step=Fail("CreateSolutionVersionTaskFailed")
))

#### 캠페인 생성

In [ ]:
lambda_create_campaign = LambdaStep(
    state_id="create campaign",
    parameters={  
        "FunctionName": "stepfunction_getsolution_metric_create_campaign", 
        "Payload": {  
            #"solution_version_arn.$": '$.Payload.solution_version_arn'  
            "solution_version_arn.$": '$.solution_version_arn'
        }
    },
    result_path = '$'
)

lambda_create_campaign.add_retry(Retry(
    error_equals=["States.TaskFailed"],
    interval_seconds=5,
    max_attempts=1,
    backoff_rate=4.0
))

lambda_create_campaign.add_catch(Catch(
    error_equals=["States.TaskFailed"],
    next_step=Fail("CreateCampaignTaskFailed")
))

## 대기-상태

#### Step Functions의 `Wait` 상태는 일정 시간 동안 대기합니다. AWS Step Functions Data Science SDK 설명서에서 [대기](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/states.html#stepfunctions.steps.states.Wait)를 참조하세요.

#### 스키마가 준비될 때까지 대기

In [ ]:
wait_state_schema = Wait(
    state_id="Wait for create schema - 5 secs",
    seconds=5
)

#### 데이터 세트 그룹이 준비될 때까지 대기

In [ ]:
wait_state_datasetgroup = Wait(
    state_id="Wait for datasetgroup - 30 secs",
    seconds=30
)

#### 데이터 세트가 준비될 때까지 대기

In [ ]:
wait_state_dataset = Wait(
    state_id="wait for dataset - 30 secs",
    seconds=30
)

#### 데이터 세트 가져오기 작업이 활성 상태가 될 때까지 대기

In [ ]:
wait_state_datasetimportjob = Wait(
    state_id="Wait for datasetimportjob - 30 secs",
    seconds=30
)

#### 레시피가 준비될 때까지 대기

In [ ]:
wait_state_receipe = Wait(
    state_id="Wait for receipe - 30 secs",
    seconds=30
)

#### 솔루션 버전이 활성 상태가 될 때까지 대기

In [ ]:
wait_state_solutionversion = Wait(
    state_id="Wait for solution version - 60 secs",
    seconds=60
)

#### 캠페인이 활성 상태가 될 때까지 대기

In [ ]:
wait_state_campaign = Wait(
    state_id="Wait for Campaign - 30 secs",
    seconds=30
)



### Lambda 작업의 상태를 확인하고 그에 따라 조치를 취합니다.

#### 상태가 실패하면 `Fail` 상태로 넘어갑니다. AWS Step Functions Data Science SDK 설명서에서 [실패](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/states.html#stepfunctions.steps.states.Fail)를 참조하세요.

### 데이터 세트 그룹 상태 확인

In [ ]:
lambda_state_datasetgroupstatus = LambdaStep(
    state_id="check dataset Group status",
    parameters={  
        "FunctionName": "stepfunction_waitforDatasetGroup", #replace with the name of the function you created
        "Payload": {  
           "input.$": '$.Payload.datasetGroupArn',
           "schemaArn.$": '$.Payload.schemaArn'
        }
    },
    result_path = '$'
)

lambda_state_datasetgroupstatus.add_retry(Retry(
    error_equals=["States.TaskFailed"],
    interval_seconds=5,
    max_attempts=1,
    backoff_rate=4.0
))

lambda_state_datasetgroupstatus.add_catch(Catch(
    error_equals=["States.TaskFailed"],
    next_step=Fail("DatasetGroupStatusTaskFailed")
))

### 데이터 세트 가져오기 작업 상태 확인

In [ ]:
lambda_state_datasetimportjob_status = LambdaStep(
    state_id="check dataset import job status",
    parameters={  
        "FunctionName": "stepfunction_waitfordatasetimportjob", #replace with the name of the function you created
        "Payload": {  
           "dataset_import_job_arn.$": '$.Payload.dataset_import_job_arn',
           "datasetGroupArn.$": '$.Payload.datasetGroupArn'
        }
    },
    result_path = '$'
)

lambda_state_datasetimportjob_status.add_retry(Retry(
    error_equals=["States.TaskFailed"],
    interval_seconds=5,
    max_attempts=1,
    backoff_rate=4.0
))

lambda_state_datasetimportjob_status.add_catch(Catch(
    error_equals=["States.TaskFailed"],
    next_step=Fail("DatasetImportJobStatusTaskFailed")
))

### 솔루션 버전 상태 확인

In [ ]:

solutionversion_succeed_state = Succeed(
    state_id="The Solution Version ready?"
)

In [ ]:
lambda_state_solutionversion_status = LambdaStep(
    state_id="check solution version status",
    parameters={  
        "FunctionName": "stepfunction_waitforSolutionVersion", #replace with the name of the function you created
        "Payload": {  
           "solution_version_arn.$": '$.Payload.solution_version_arn'           
        }
    },
    result_path = '$'
)

lambda_state_solutionversion_status.add_retry(Retry(
    error_equals=["States.TaskFailed"],
    interval_seconds=5,
    max_attempts=1,
    backoff_rate=4.0
))

lambda_state_solutionversion_status.add_catch(Catch(
    error_equals=["States.TaskFailed"],
    next_step=Fail("SolutionVersionStatusTaskFailed")
))

### 캠페인 상태 확인

In [ ]:
lambda_state_campaign_status = LambdaStep(
    state_id="check campaign status",
    parameters={  
        "FunctionName": "stepfunction_waitforCampaign", #replace with the name of the function you created
        "Payload": {  
           "campaign_arn.$": '$.Payload.campaign_arn'           
        }
    },
    result_path = '$'
)

lambda_state_campaign_status.add_retry(Retry(
    error_equals=["States.TaskFailed"],
    interval_seconds=5,
    max_attempts=1,
    backoff_rate=4.0
))

lambda_state_campaign_status.add_catch(Catch(
    error_equals=["States.TaskFailed"],
    next_step=Fail("CampaignStatusTaskFailed")
))

## 선택-상태

이제 앞서 생성한 선택 상태에 분기를 연결합니다. [AWS Step Functions Data Science SDK 설명서](https://aws-step-functions-data-science-sdk.readthedocs.io)에서 선택 규칙을 참조하세요.

#### 워크플로 경로를 정의하기 위한 단계를 하나로 연결합니다.

다음 셀은 위에서 생성한 단계를 순차적 그룹으로 연결합니다. 새 경로에는 앞서 생성한 Lambda 상태, 대기 상태 및 성공 상태가 순차적으로 포함됩니다.

#### 워크플로 경로의 단계를 하나로 연결한 후에는 워크플로를 정의하고 시각화합니다.

In [ ]:
create_campaign_choice_state = Choice(
    state_id="Is the Campaign ready?"
)

In [ ]:
create_campaign_choice_state.add_choice(
    rule=ChoiceRule.StringEquals(variable=lambda_state_campaign_status.output()['Payload']['status'], value='ACTIVE'),
    next_step=Succeed("CampaignCreatedSuccessfully")     
)
create_campaign_choice_state.add_choice(
    ChoiceRule.StringEquals(variable=lambda_state_campaign_status.output()['Payload']['status'], value='CREATE PENDING'),
    next_step=wait_state_campaign
)
create_campaign_choice_state.add_choice(
    ChoiceRule.StringEquals(variable=lambda_state_campaign_status.output()['Payload']['status'], value='CREATE IN_PROGRESS'),
    next_step=wait_state_campaign
)

create_campaign_choice_state.default_choice(next_step=Fail("CreateCampaignFailed"))


In [ ]:
solutionversion_choice_state = Choice(
    state_id="Is the Solution Version ready?"
)

In [ ]:
solutionversion_succeed_state = Succeed(
    state_id="The Solution Version ready?"
)

In [ ]:
solutionversion_choice_state.add_choice(
    rule=ChoiceRule.StringEquals(variable=lambda_state_solutionversion_status.output()['Payload']['status'], value='ACTIVE'),
    next_step=solutionversion_succeed_state   
)
solutionversion_choice_state.add_choice(
    ChoiceRule.StringEquals(variable=lambda_state_solutionversion_status.output()['Payload']['status'], value='CREATE PENDING'),
    next_step=wait_state_solutionversion
)
solutionversion_choice_state.add_choice(
    ChoiceRule.StringEquals(variable=lambda_state_solutionversion_status.output()['Payload']['status'], value='CREATE IN_PROGRESS'),
    next_step=wait_state_solutionversion
)

solutionversion_choice_state.default_choice(next_step=Fail("create_solution_version_failed"))


In [ ]:
datasetimportjob_succeed_state = Succeed(
    state_id="The Solution Version ready?"
)

In [ ]:
datasetimportjob_choice_state = Choice(
    state_id="Is the DataSet Import Job ready?"
)

In [ ]:
datasetimportjob_choice_state.add_choice(
    rule=ChoiceRule.StringEquals(variable=lambda_state_datasetimportjob_status.output()['Payload']['status'], value='ACTIVE'),
    next_step=datasetimportjob_succeed_state   
)
datasetimportjob_choice_state.add_choice(
    ChoiceRule.StringEquals(variable=lambda_state_datasetimportjob_status.output()['Payload']['status'], value='CREATE PENDING'),
    next_step=wait_state_datasetimportjob
)
datasetimportjob_choice_state.add_choice(
    ChoiceRule.StringEquals(variable=lambda_state_datasetimportjob_status.output()['Payload']['status'], value='CREATE IN_PROGRESS'),
    next_step=wait_state_datasetimportjob
)


datasetimportjob_choice_state.default_choice(next_step=Fail("dataset_import_job_failed"))


In [ ]:
datasetgroupstatus_choice_state = Choice(
    state_id="Is the DataSetGroup ready?"
)

## 워크플로

### 워크플로 정의

다음 셀에서는 워크플로에서 사용할 단계를 정의합니다. 그런 다음 워크플로를 생성하고 시각화하고 실행합니다.

단계는 AWS Step Functions의 상태와 관련이 있습니다. 자세한 내용은 AWS Step Functions 개발자 가이드에서 [상태](https://docs.aws.amazon.com/step-functions/latest/dg/concepts-states.html)를 참조하세요. AWS Step Functions Data Science SDK API에 대한 자세한 내용은 https://aws-step-functions-data-science-sdk.readthedocs.io를 참조하세요.




### 데이터 세트 워크플로

In [ ]:
Dataset_workflow_definition=Chain([lambda_state_schema,
                                   wait_state_schema,
                                   lambda_state_datasetgroup,
                                   wait_state_datasetgroup,
                                   lambda_state_datasetgroupstatus
                                  ])

In [ ]:
Dataset_workflow = Workflow(
    name="Dataset-workflow",
    definition=Dataset_workflow_definition,
    role=workflow_execution_role
)

In [ ]:
Dataset_workflow.render_graph()

In [ ]:
DatasetWorkflowArn = Dataset_workflow.create()

### 데이터 세트 가져오기 워크플로

In [ ]:
DatasetImport_workflow_definition=Chain([lambda_state_createdataset,
                                   wait_state_dataset,
                                   lambda_state_datasetimportjob,
                                   wait_state_datasetimportjob,
                                   lambda_state_datasetimportjob_status,
                                   datasetimportjob_choice_state
                                  ])

In [ ]:
DatasetImport_workflow = Workflow(
    name="DatasetImport-workflow",
    definition=DatasetImport_workflow_definition,
    role=workflow_execution_role
)

In [ ]:
DatasetImport_workflow.render_graph()

In [ ]:
DatasetImportflowArn = DatasetImport_workflow.create()

레시피 및 솔루션 워크플로

In [ ]:
Create_receipe_sol_workflow_definition=Chain([lambda_state_select_receipe_create_solution,
                                   wait_state_receipe,
                                   lambda_create_solution_version,
                                   wait_state_solutionversion,
                                   lambda_state_solutionversion_status,
                                   solutionversion_choice_state
                                  ])

In [ ]:
Create_receipe_sol_workflow = Workflow(
    name="Create_receipe_sol-workflow",
    definition=Create_receipe_sol_workflow_definition,
    role=workflow_execution_role
)

In [ ]:
Create_receipe_sol_workflow.render_graph()

In [ ]:
CreateReceipeArn = Create_receipe_sol_workflow.create()

캠페인 생성 워크플로

In [ ]:
Create_Campaign_workflow_definition=Chain([lambda_create_campaign,
                                   wait_state_campaign,
                                   lambda_state_campaign_status,
                                   wait_state_datasetimportjob,
                                   create_campaign_choice_state
                                  ])

In [ ]:
Campaign_workflow = Workflow(
    name="Campaign-workflow",
    definition=Create_Campaign_workflow_definition,
    role=workflow_execution_role
)

In [ ]:
Campaign_workflow.render_graph()

In [ ]:
CreateCampaignArn = Campaign_workflow.create()

기본 워크플로

In [ ]:
call_dataset_workflow_state = Task(
    state_id="DataSetWorkflow",
    resource="arn:aws:states:::states:startExecution.sync:2",
    parameters={
                                "Input": "true",
                                #"StateMachineArn": "arn:aws:states:us-east-1:444602785259:stateMachine:Dataset-workflow",
                                "StateMachineArn": DatasetWorkflowArn
                }
)

In [ ]:
call_datasetImport_workflow_state = Task(
    state_id="DataSetImportWorkflow",
    resource="arn:aws:states:::states:startExecution.sync:2",
    parameters={
                                 "Input":{
                                    "schemaArn.$": "$.Output.Payload.schemaArn",
                                    "datasetGroupArn.$": "$.Output.Payload.datasetGroupArn"
                                   },
                                "StateMachineArn": DatasetImportflowArn,
                }
)

In [ ]:
call_receipe_solution_workflow_state = Task(
    state_id="ReceipeSolutionWorkflow",
    resource="arn:aws:states:::states:startExecution.sync:2",
    parameters={
                                 "Input":{
                                    "datasetGroupArn.$": "$.Output.Payload.datasetGroupArn"

                                   },
                                "StateMachineArn": CreateReceipeArn
                }
)

In [ ]:
call_campaign_solution_workflow_state = Task(
    state_id="CampaignWorkflow",
    resource="arn:aws:states:::states:startExecution.sync:2",
    parameters={
                                 "Input":{
                                    "solution_version_arn.$": "$.Output.Payload.solution_version_arn"

                                   },
                                "StateMachineArn": CreateCampaignArn
                }
)

In [ ]:
Main_workflow_definition=Chain([call_dataset_workflow_state,
                                call_datasetImport_workflow_state,
                                call_receipe_solution_workflow_state,
                                call_campaign_solution_workflow_state
                               ])

In [ ]:
Main_workflow = Workflow(
    name="Main-workflow",
    definition=Main_workflow_definition,
    role=workflow_execution_role
)

In [ ]:
Main_workflow.render_graph()

In [ ]:
Main_workflow.create()

In [ ]:
Main_workflow_execution = Main_workflow.execute()

Main_workflow_execution = Workflow(
    name="Campaign_Workflow",
    definition=path1,
    role=workflow_execution_role
)


In [ ]:
#Main_workflow_execution.render_graph()

### 워크플로 생성 및 실행

다음 셀에서는 AWS Step Functions에서 [create](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/workflow.html#stepfunctions.workflow.Workflow.create)를 사용하여 분기하는 행복 워크플로를 생성하고 [execute](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/workflow.html#stepfunctions.workflow.Workflow.execute)를 사용하여 실행합니다.


In [ ]:
#personalize_workflow.create()

In [ ]:
#personalize_workflow_execution = happy_workflow.execute()

###  워크플로 진행 검토

[render_progress](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/workflow.html#stepfunctions.workflow.Execution.render_progress)를 사용하여 워크플로 진행을 검토합니다.

[list_events](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/workflow.html#stepfunctions.workflow.Execution.list_events)를 호출하여 워크플로 실행의 모든 이벤트를 나열하면서 실행 기록을 검토합니다.

In [ ]:
Main_workflow_execution.render_progress()

In [ ]:
Main_workflow_execution.list_events(html=True)

## 생성-추천

### 이제 캠페인을 생성했으므로 캠페인에 대한 추천을 생성해 보겠습니다.

#### 사용자 및 항목 선택

In [ ]:
items = pd.read_csv('./ml-100k/u.item', sep='|', usecols=[0,1], encoding='latin-1')
items.columns = ['ITEM_ID', 'TITLE']


user_id, item_id, rating, timestamp = data.sample().values[0]

user_id = int(user_id)
item_id = int(item_id)

print("user_id",user_id)
print("items",items)


item_title = items.loc[items['ITEM_ID'] == item_id].values[0][-1]
print("USER: {}".format(user_id))
print("ITEM: {}".format(item_title))
print("ITEM ID: {}".format(item_id))


In [ ]:
wait_recommendations = Wait(
    state_id="Wait for recommendations - 10 secs",
    seconds=10
)

#### Lambda 작업

In [ ]:
lambda_state_get_recommendations = LambdaStep(
    state_id="get recommendations",
    parameters={  
        "FunctionName": "stepfunction_getRecommendations", 
        "Payload": {  
           "campaign_arn": 'arn:aws:personalize:us-east-1:261602857181:campaign/stepfunction-campaign',            
           "user_id": user_id,  
           "item_id": item_id             
        }
    },
    result_path = '$'
)

lambda_state_get_recommendations.add_retry(Retry(
    error_equals=["States.TaskFailed"],
    interval_seconds=5,
    max_attempts=1,
    backoff_rate=4.0
))

lambda_state_get_recommendations.add_catch(Catch(
    error_equals=["States.TaskFailed"],
    next_step=Fail("GetRecommendationTaskFailed")
    #next_step=recommendation_path   
))

#### 성공 상태 생성

In [ ]:
workflow_complete = Succeed("WorkflowComplete")

In [ ]:
recommendation_path = Chain([ 
lambda_state_get_recommendations,
wait_recommendations,
workflow_complete
])

### 추천 워크플로 정의, 생성, 렌더링 및 실행

다음 셀에서는 AWS Step Functions에서 [create](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/workflow.html#stepfunctions.workflow.Workflow.create)를 사용하여 워크플로를 생성하고 [execute](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/workflow.html#stepfunctions.workflow.Workflow.execute)를 사용하여 실행합니다.

In [ ]:
recommendation_workflow = Workflow(
    name="Recommendation_Workflow4",
    definition=recommendation_path,
    role=workflow_execution_role
)



In [ ]:
recommendation_workflow.render_graph()

In [ ]:
recommendation_workflow.create()

In [ ]:
recommendation_workflow_execution = recommendation_workflow.execute()

### 진행 검토

[render_progress](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/workflow.html#stepfunctions.workflow.Execution.render_progress)를 사용하여 워크플로 진행을 검토합니다.

[list_events](https://aws-step-functions-data-science-sdk.readthedocs.io/en/latest/workflow.html#stepfunctions.workflow.Execution.list_events)를 호출하여 워크플로 실행의 모든 이벤트를 나열하면서 실행 기록을 검토합니다.

In [ ]:
recommendation_workflow_execution.render_progress()

In [ ]:
recommendation_workflow_execution.list_events(html=True)


In [ ]:
item_list = recommendation_workflow_execution.get_output()['Payload']['item_list']

### 추천 받기

In [ ]:
item_list = recommendation_workflow_execution.get_output()['Payload']['item_list']

print("Recommendations:")
for item in item_list:
    np.int(item['itemId'])
    item_title = items.loc[items['ITEM_ID'] == np.int(item['itemId'])].values[0][-1]
    print(item_title)


## Amazon Personalize 리소스 정리

Amazon Personalize 및 상태 시스템에서 생성된 블로그를 정리해야 합니다. Amazon Personalize 콘솔에 로그인하고 데이터 세트 그룹, 데이터 세트, 솔루션, 레시피, 캠페인 등의 리소스를 삭제합니다. 

## 상태 시스템 리소스 정리

In [ ]:
Campaign_workflow.delete()

recommendation_workflow.delete()

Main_workflow.delete()

Create_receipe_sol_workflow.delete()

DatasetImport_workflow.delete()

Dataset_workflow.delete()
